In [5]:
!python preprocess.py \
    --use-default-paths \
    --scrape-studies \
    --scrape-spreadsheet \



Scraping studies...
100%|█████████████████████████████████████| 12259/12259 [23:18<00:00,  8.76it/s]
Fetched studies. Checking integrity...
Saving studies...
Raw studies saved.



Scraping spreadsheet data...
Verifying URLs...
100%|█████████████████████████████████████████| 488/488 [00:12<00:00, 37.85it/s]
Removing 393 invalid URLS
Scraping and extracting PDFs...
 35%|██████████████▉                            | 33/95 [00:14<01:13,  1.18s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
100%|███████████████████████████████████████████| 95/95 [01:35<00:00,  1.01s/it]
Scraped spreadsheet data. Saving...
Spreadsheet data saved.



Using default paths:
Irrelevant: ../../data/unprocessed/irrelevant/
Spreadsheet: ../../data/unprocessed/spreadsheet.json
Synopses: ../../data/unprocessed/synopses/
Studies: ../../data/unprocessed/studies.json



Cleaning irrelevant data.
Load

In [6]:
!python train.py \
    --model CuML \
    --data-path='../../data/level-0.5/data.json' \
    --output-path='./models/level-1/cuML_classifier.pkl' \
    --timer



Loading data...

/workspace/GreyLit/venv/lib/python3.10/site-packages/cudf/io/json.py:108: UserWarning: Using CPU via Pandas to read JSON dataset, this may be GPU accelerated in the future
  warnings.warn(
Data loaded. 
Splitting data...
Data split.

Initialising CuML trainer...

Train data:
<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 15088 entries, 0 to 15087
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   url           15088 non-null  object
 1   text          15088 non-null  object
 2   relevance     15088 non-null  object
 3   multiclasses  15088 non-null  list
dtypes: list(1), object(3)
memory usage: 1017.3+ MB

Test data:
<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 3773 entries, 15088 to 18860
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   url           3773 non-null   object
 1   text          3773 non-null   object
 2   

In [7]:
!python predict.py \
    --model='CuML' \
    --model-path='./models/level-1/cuML_classifier.pkl' \
    --data-path='../../data/level-0.5/irrelevant.json' \
    --output-path='../../results/' \
    --save-top 200 \
    --timer



Loading model from ./models/level-1/cuML_classifier.pkl...
Model loaded.

Loading data...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10181 entries, 0 to 10180
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   url           10181 non-null  object
 1   text          10181 non-null  object
 2   relevance     10181 non-null  object
 3   multiclasses  10181 non-null  object
dtypes: object(4)
memory usage: 318.3+ KB

Data loaded.

Generating classification predictions...

Classifying 10181 files.
Estimated time:  1.0  minutes  3  seconds


Testing time on 10181 articles:  60.18002104759216  seconds

Files processed per second: 169.17574674738916
Evaluating classifier...

prediction
irrelevant    10162
relevant         19
Name: count, dtype: int64
Accuracy:  0.9981337786072095
Precision:  0.5
Recall:  0.49906688930360477
F1:  0.4995330088974094
Confusion matrix: 
 [[10162    19]
 [    0     0]]


Potential new Con

In [8]:
!python train.py \
    --model FastFit \
    --data-path='../../data/level-0.5/data.json' \
    --embedding-model='avsolatorio/GIST-Embedding-v0' \
    --output-path='./models/level-2/' \
    --chunk-size 512 \
    --batch-size 32 \
    --samples-per-label 100000 \
    --timer

    # TODO: Fix training with Embedding models



Loading data...
Data loaded. 
Splitting data...
Data split.

Chunking data...
100%|█████████████████████████████████████| 3018/3018 [00:01<00:00, 2754.55it/s]
Data chunked.

Downsampling data...
Data downsampled.

Converting data to FastFit format...

Train data:
Dataset({
    features: ['chunk_id', 'url', 'text', 'relevance', 'multiclasses'],
    num_rows: 200000
})

Test data:
Dataset({
    features: ['chunk_id', 'url', 'text', 'relevance', 'multiclasses', '__index_level_0__'],
    num_rows: 200
})

Initialising FastFit trainer...
08/16/2024 16:07:21 - WARNING - fastfit.train - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: True
Running tokenizer on dataset to infer max length for both query and document: 10
Running tokenizer on dataset to infer max length for both query and document: 10
Running tokenizer on dataset to infer max length for both query and document: 10
Running tokenizer on dataset: 100%|█| 200000/200000 [03:31<00:00, 945.97 exam

In [9]:
!python predict.py \
    --model='FastFit' \
    --model-path='./models/level-2/avsolatorio/GIST-Embedding-v0' \
    --data-path='../../data/level-1.5/potential.json' \
    --output-path='../../results/level-2/' \
    --save-top 200 \
    --timer

Loading model from ./models/level-2/avsolatorio/GIST-Embedding-v0 ...
Model loaded.
Traceback (most recent call last):
  File "/workspace/GreyLit/venv/lib/python3.10/site-packages/transformers/models/auto/configuration_auto.py", line 982, in from_pretrained
    config_class = CONFIG_MAPPING[config_dict["model_type"]]
  File "/workspace/GreyLit/venv/lib/python3.10/site-packages/transformers/models/auto/configuration_auto.py", line 684, in __getitem__
    raise KeyError(key)
KeyError: 'FastFit'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/workspace/GreyLit/GreyLiteratureClassifier/src/scripts/predict.py", line 191, in <module>
    main(**vars(args))
  File "/workspace/GreyLit/GreyLiteratureClassifier/src/scripts/predict.py", line 165, in main
    test_embeddings(model_path, data_path, output_path, seed, save_top, timer, **kwargs)
  File "/workspace/GreyLit/GreyLiteratureClassifier/src/scripts/predict.py", line 91, in te

In [10]:
!python predict.py \
    --model='FastFit' \
    --model-path='../notebooks/FastFit/level-1/models/relevance/avsolatorio/GIST-Embedding-v0' \
    --data-path='../../data/level-1.5/potential.json' \
    --output-path='../../results/level-2/notebook_pretrained/' \
    --save-top 200 \
    --timer

Loading model from ../notebooks/FastFit/level-1/models/relevance/avsolatorio/GIST-Embedding-v0 ...
Model loaded.
Tokenizer loaded with max_length: 512
The model 'FastFit' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForS